<a href="https://colab.research.google.com/github/deltorobarba/astrophysics/blob/master/vqe_ising.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Variational Quantum Eigensolver to optimize an Ising model**

Code example: Use the variational quantum eigensolver in Cirq to optimize an Ising model. This demonstrates the four key components of VQE: defining the Hamiltonian (the problem to solve), creating a parameterized quantum circuit (the Ansatz), computing the expectation value of the Hamiltonian, and optimizing the parameters to find the ground state energy.

In [1]:
!pip install cirq -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.7/532.7 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.3/69.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

In [2]:
import cirq
import numpy as np
import scipy.optimize

# 1. Define the Hamiltonian (Ising Model)
def ising_hamiltonian(qubits, J=1.0, h=0.5):
    """Creates an Ising model Hamiltonian."""
    hamiltonian = cirq.PauliSum()
    for i in range(len(qubits)):
        hamiltonian += -J * cirq.Z(qubits[i]) * cirq.Z(qubits[(i + 1) % len(qubits)])  # Periodic boundary
        hamiltonian += -h * cirq.X(qubits[i])  # Using X instead of Z for transverse field
    return hamiltonian

# 2. Define the Ansatz (Parameterized Quantum Circuit)
def ansatz(qubits, params):
    """Creates a simple hardware-efficient ansatz."""
    circuit = cirq.Circuit()

    # First layer of rotations
    for i, qubit in enumerate(qubits):
        circuit.append(cirq.rx(params[2 * i])(qubit))
        circuit.append(cirq.rz(params[2 * i + 1])(qubit))

    # Entangling layer
    for i in range(len(qubits)):
        circuit.append(cirq.CZ(qubits[i], qubits[(i + 1) % len(qubits)]))

    # Second layer of rotations
    offset = 2 * len(qubits)
    for i, qubit in enumerate(qubits):
        circuit.append(cirq.rx(params[offset + 2 * i])(qubit))
        circuit.append(cirq.rz(params[offset + 2 * i + 1])(qubit))

    return circuit

# 3. Calculate the Expectation Value
def expectation_value(qubits, params, hamiltonian, simulator):
    """Calculates the expectation value of the Hamiltonian."""
    circuit = ansatz(qubits, params)

    # The key fix: create a proper qubit-to-index mapping
    qubit_map = {qubit: i for i, qubit in enumerate(qubits)}

    result = simulator.simulate(circuit)
    state_vector = result.final_state_vector

    return hamiltonian.expectation_from_state_vector(state_vector, qubit_map).real

# 4. Optimization
def vqe_optimization(num_qubits, simulator, max_iterations=100):
    """Performs VQE optimization."""
    qubits = cirq.LineQubit.range(num_qubits)
    hamiltonian = ising_hamiltonian(qubits)

    # 4 parameters per qubit (2 rotation angles per layer × 2 layers)
    initial_params = np.random.uniform(0, 2 * np.pi, 4 * num_qubits)

    def cost_function(params):
        return expectation_value(qubits, params, hamiltonian, simulator)

    result = scipy.optimize.minimize(
        cost_function,
        initial_params,
        method='COBYLA',
        options={'maxiter': max_iterations}
    )

    # Return both the optimized energy and parameters
    return result.fun, result.x

# Example Usage
def main():
    num_qubits = 4
    simulator = cirq.Simulator()

    print(f"Running VQE for {num_qubits} qubits...")
    ground_state_energy, optimal_params = vqe_optimization(num_qubits, simulator)

    print(f"Estimated Ground State Energy: {ground_state_energy}")

    # Optional: Visualize the optimal circuit
    qubits = cirq.LineQubit.range(num_qubits)
    optimal_circuit = ansatz(qubits, optimal_params)
    print("\nOptimal Circuit:")
    print(optimal_circuit)

if __name__ == "__main__":
    main()

Running VQE for 4 qubits...
Estimated Ground State Energy: -4.23966670781374

Optimal Circuit:
                                                    ┌──────────┐
0: ───Rx(1.02π)────Rz(1.74π)────@────────────────────@─────────────Rx(-1.9π)────Rz(-0.456π)───
                                │                    │
1: ───Rx(0.562π)───Rz(1.93π)────@───@───Rx(1.61π)────┼Rz(1.77π)───────────────────────────────
                                    │                │
2: ───Rx(0.008π)───Rz(-1.81π)───────@───@────────────┼Rx(1.08π)────Rz(1.58π)──────────────────
                                        │            │
3: ───Rx(1.83π)────Rz(1.84π)────────────@────────────@─────────────Rx(0.824π)───Rz(0.909π)────
                                                    └──────────┘
